In [4]:
import pandas as pd
from pathlib import Path

# Define file paths for uploaded files
gdp_file = "Cleaned Data Csv/Canada_USA_GDP_2014-2024.csv"
wti_file = "Cleaned Data Csv/WTI_data_year_only_2014_2024.csv"

# Load datasets
gdp_df = pd.read_csv(gdp_file)
wti_df = pd.read_csv(wti_file)

# Remove unnecessary columns (such as unnamed index columns if present)
gdp_df = gdp_df.loc[:, ~gdp_df.columns.str.contains('Unnamed', case=False, na=False)]
wti_df = wti_df.loc[:, ~wti_df.columns.str.contains('Unnamed', case=False, na=False)]

# Standardize column names by stripping spaces and converting to lowercase
gdp_df.columns = gdp_df.columns.str.strip().str.lower()
wti_df.columns = wti_df.columns.str.strip().str.lower()

# Ensure 'year' column exists and convert to integer
if "year" in gdp_df.columns and "year" in wti_df.columns:
    gdp_df["year"] = pd.to_numeric(gdp_df["year"], errors="coerce").astype("Int64")
    wti_df["year"] = pd.to_numeric(wti_df["year"], errors="coerce").astype("Int64")

# Clean WTI percent change column (remove '%' and convert to float)
if "percentchange" in wti_df.columns:
    wti_df["percentchange"] = (
        wti_df["percentchange"]
        .str.replace("%", "", regex=True)
        .astype(float)
    )

# Rename columns for clarity (if they exist)
gdp_df.rename(
    columns={"gdp per capita (constant 2015 us$)": "gdp_per_capita"}, inplace=True
)
wti_df.rename(
    columns={"price": "wti_price", "percentchange": "wti_percent_change"}, inplace=True
)

# Remove duplicates
gdp_df.drop_duplicates(inplace=True)
wti_df.drop_duplicates(inplace=True)

# Drop any remaining NaN values
gdp_df.dropna(inplace=True)
wti_df.dropna(inplace=True)

# Merge cleaned datasets on the year column
cleaned_merged_df = gdp_df.merge(wti_df, on="year", how="inner")

# Save cleaned dataset to a CSV file for download
cleaned_file_path = "Cleaned Data Csv/Cleaned_GDP_WTI_Data.csv"
cleaned_merged_df.to_csv(cleaned_file_path, index=False)

# Provide download link
cleaned_file_path

'Cleaned Data Csv/Cleaned_GDP_WTI_Data.csv'

In [5]:
cleaned_merged_df

,country name,country code,year,gdp_per_capita,wti_price,wti_percent_change
0,Canada,CAN,2014,43643.24,97.49,-0.94
1,United States,USA,2014,55817.56,97.49,-0.94
2,Canada,CAN,2015,43594.19,48.24,-9.44
3,United States,USA,2015,57040.21,48.24,-9.44
4,Canada,CAN,2016,43551.34,33.62,-9.23
5,United States,USA,2016,57658.67,33.62,-9.23
6,Canada,CAN,2017,44339.39,52.81,-1.69
7,United States,USA,2017,58703.14,52.81,-1.69
8,Canada,CAN,2018,44907.34,64.73,7.13
9,United States,USA,2018,60127.21,64.73,7.13


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = cleaned_merged_df.copy()
X = df.drop(columns=['wti_percent_change']) 
y = df['wti_percent_change']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training set:", X_train.shape, y_train.shape)
print("Testing set:", X_test.shape, y_test.shape)

Training set: (16, 5) (16,)
Testing set: (4, 5) (4,)


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Copy the cleaned DataFrame
df = cleaned_merged_df.copy()

# Identify categorical columns
categorical_cols = ['country name', 'country code']  # Removed 'type_' as it's not in your dataset

# One-Hot Encode categorical columns
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Define Features (X) and Target (y)
X = df.drop(columns=['wti_percent_change'])  # Features
y = df['wti_percent_change']  # Target variable

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()

# Fit the scaler on training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test set
X_test_scaled = scaler.transform(X_test)

# Display the shape of transformed datasets
print("Scaled Training set shape:", X_train_scaled.shape)
print("Scaled Testing set shape:", X_test_scaled.shape)


Scaled Training set shape: (16, 5)
Scaled Testing set shape: (4, 5)


Train and Evaluate the Model

In [8]:
import tensorflow as tf

# Get number of input features (columns in X_train)
number_input_features = X_train_scaled.shape[1]  

# Define the number of nodes for each hidden layer
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50

# Initialize the neural network
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer for regression (WTI price change prediction)
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))  # Linear for regression

# Check model architecture
nn.summary()

# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae"])

# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae = nn.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")


2025-02-28 21:47:05.583122: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jsflora/Library/Python/3.12/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         4,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,581 (17.89 KB)

 Trainable params: 4,581 (17.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 123.7449 - mae: 9.8377 - val_loss: 100.9942 - val_mae: 7.2363
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 123.0780 - mae: 9.8191 - val_loss: 101.2830 - val_mae: 7.2362
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - loss: 122.4614 - mae: 9.8018 - val_loss: 101.5404 - val_mae: 7.2344
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 121.8616 - mae: 9.7844 - val_loss: 101.7389 - val_mae: 7.2298
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 121.3020 - mae: 9.7679 - val_loss: 101.9517 - val_mae: 7.2263
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 120.7635 - mae: 9.7520 - val_loss: 102.1618 - val_mae: 7.2229
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 120.2557 - mae: 9.7372 - val_loss: 102.3655 - val_mae: 7.2196
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 119.7616 - mae: 9.7232 - val_loss: 102.5616 - val_mae: 7.2171
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━